In [1]:
from src import Color
from src.material.material import create_phong_material
from src import World
from src import Sphere, Plane
from src import PointLight
from src import ipynb_display_image
from src import Resolution
from src import Camera
from src import Vertex, Vector
from src import Scene, RenderMethod, ShadingModel, QualityPreset
from src.scene.animation.animator import *
from src.shading import *

# Simple scene with a sphere and a plane and a light source
no skybox just to demonstrate basic functionality

In [2]:
# colors
brown = Color.custom_rgb(150, 75, 0)
red = Color.custom_rgb(255, 0, 0)

# materials
glossy_red = create_phong_material(name="glossy_red", base_color=red, spec_color=Color.custom_rgb(255, 255, 255), shininess=100, ior=1.0, reflectivity=0.3, transparency=0.0)
mate_blue = create_phong_material(name="mate_blue", base_color=Color.custom_rgb(0, 0, 255), spec_color=Color.custom_rgb(128, 128, 128), shininess=10, ior=1.0, reflectivity=0.05, transparency=0.0)

# objects
sphere = Sphere(center=Vertex(0, 0.2, -5), radius=1.0, material=glossy_red)
plane = Plane(point=Vertex(0, -1, 0), normal=Vector(0, 1, 0), material=mate_blue)

# world
world = World()
world.add(sphere)
world.add(plane)

# lights
point_light = PointLight(position=Vertex(5, 5, 0), intensity=2000.0, falloff=0.01)

# scene setup
camera = Camera(
    fov = 45,
    resolution = Resolution.R360p,
    origin = Vertex(0, 1.0, 2.0),
    direction = Vector(0, -0.2, -1),
)


scene = Scene(
    camera = camera,
    world = world,
    lights = [point_light],
    skybox_path = None
)

scene.validate()

Scene validation passed.


# Now we can render the scene

### Preview

In [3]:
#prewiew render with no antialiasing, no recursion and no skybox at 144p resolution for speed just to see all objects in the scene

#scene.render_preview()

Render

In [ ]:
animation_setup = AnimationSetup(
    # camera move
    move_from = Vertex(0, 1.0, 2.0),
    move_to   = Vertex(-6, 0.1, 0.5),
    move_start_delay = 0.0,
    move_duration    = 2.0,

    # camera rotate yaw/pitch/roll
    rotate_axis      = Vector(-0.2, 1, 0.2),
    rotate_angle_deg = -55.0,
    rotate_start_delay = 0.0,
    rotate_duration    = 2.0,

    # zoom in
    zoom_from       = 80.0,
    zoom_to         = 30.0,
    zoom_start_delay = 0.0,
    zoom_duration    = 2.0,
)


animator = Animator(
    animation_setup=animation_setup,
    animation_fps=24,
    animation_length_seconds=2.0,
    animation_resolution=Resolution.R360p,
)
animator.animate_to_mp4(
    scene=scene,
    ease=EaseType.EASE_IN_OUT
)



Rendering frame 1/48
At time 0.00s of 2.00s - frame duration 0.0417s - Percent: 2.08%
Using skybox: None
Rendering fast at resolution Resolution.R360p with FOV 80.0 and samples_per_pixel=1, max_depth=5
No progress bar in multithreaded mode - switch to other render method for that if needed - this is supposed to be fast!
------------------------------------------------------------
Using 8 CPU cores for rendering.
------------------------------------------------------------
Fast render saved to /Users/jansik/PycharmProjects/Educational-Ray-Tracer/notebooks/animation_frames/frame_0000.png
Rendering frame 2/48
At time 0.04s of 2.00s - frame duration 0.0417s - Percent: 4.17%
Using skybox: None
Rendering fast at resolution Resolution.R360p with FOV 79.95659722222223 and samples_per_pixel=1, max_depth=5
No progress bar in multithreaded mode - switch to other render method for that if needed - this is supposed to be fast!
------------------------------------------------------------
Using 8 CPU